- Javier Concejo
- Juan Sevilla
- Yago Carbó

# Librerías

In [ ]:
import spacy
from spacy import displacy
import pandas as pd

nlp_es = spacy.load('es_core_news_lg')

import warnings
warnings.filterwarnings("ignore")

# Formas canónicas y tokens

In [ ]:
lista=list(noticias['Texto'])

def lema_canonica(texto):
    noticias_doc=nlp(texto)
    tokens_tagged = [(palabra.text, palabra.lemma_) for palabra in noticias_doc]
    df=pd.DataFrame(tokens_tagged, columns=['Palabra', 'F. Canónica'])
    return(df.iloc[:15])
lema_canonica(lista[0])

# Tokens

In [ ]:
def tokens_palabras_spacy(texto):
    texto_tratado = texto.replace("\n", " ")
    doc = nlp(texto_tratado)
    return(doc)

# Frases

In [ ]:
def Token_frases(texto):
    doc = nlp(texto)
    sentences = [sentence.text for sentence in doc.sents]
    return (sentences)

# Stopwords

In [ ]:
#Stopwords para varias filas
lista_hotel_stopwords=list(hotel["text"].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)])))

#stopwords para un texto
doc = nlp(lazarillo_text)
def stopwords:
    words = [word for word in doc if not word.is_stop]
    return(words)

# Traducción

In [ ]:
from textblob import TextBlob
texto = "In new lawsuits brought against the ride-sharing companies Uber and Lyft, the top prosecutors in Los Angeles"

def traduce(oracion, original, traducido):
    input = TextBlob(oracion)
    return(input.translate(from_lang=original, to=traducido))


traduce(texto, 'en','es')

# TIFDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

lista=list(doc_definitivo['texto'])

def tfidf_max_feature(datos):
    TFIDF_vectorizer = TfidfVectorizer(max_features=20)
    TFIDF_array=TFIDF_vectorizer.fit_transform(datos).toarray()
    vocab_TFIDF = TFIDF_vectorizer.get_feature_names()
    TFIDF=pd.DataFrame(TFIDF_array, columns=vocab_TFIDF)
    return(TFIDF.head(5))
tfidf_max_feature(lista)

# BOW

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

lista=list(doc_definitivo['texto'])

def BOW_max_feature(datos):
    BOW_vectorizer = CountVectorizer(max_features=20)
    BOW_array=BOW_vectorizer.fit_transform(datos).toarray()
    vocab_BOW = BOW_vectorizer.get_feature_names()
    BOW=pd.DataFrame(BOW_array, columns=vocab_BOW)
    return(BOW.head(5))
BOW_max_feature(lista)

# OHE

In [ ]:
from sklearn.preprocessing import Binarizer
from sklearn.feature_extraction.text import CountVectorizer

lista=list(doc_definitivo['texto'])

def OHE(datos):
    OHE_vectorizer = CountVectorizer(max_features=20)
    OHE_array=OHE_vectorizer.fit_transform(datos).toarray()
    transformer = Binarizer().fit(OHE_array)
    array2=transformer.transform(OHE_array)
    vocab = OHE_vectorizer.get_feature_names()
    OHE=pd.DataFrame(array2, columns=vocab)
    return(OHE.head(5))
OHE(lista)

# Categoría gramatical (POS) y Entidades (NER)

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
texto=sent_tokenize(texto)

docum_analizado1=nlp_es(texto[0])

tokens_1=[]
dependencia_1=[]
dep_1=[]
tag_1=[]

def dependencias(texto):
    for token in texto:
        tokens_1.append(token)
        tag_1.append(token.tag_)
        dep_1.append(token.dep_)
        dependencia_1.append(spacy.explain(token.dep_))
    
    list_of_tuples_1 = list(zip(tokens_1,tag_1,dep_1,dependencia_1))
    tabla_1 = pd.DataFrame(list_of_tuples_1,columns=['Tokens', 'Tag','Dep','Dependencia'])
    return(tabla_1)

dependencias(docum_analizado1)

In [ ]:
# diagrama con flechas
doc = nlp(texto)
for sent in doc.sents:
    displacy.render(sent, jupyter=True, options={'distance': 110,'arrow_stroke': 2,'arrow_width': 8})

In [ ]:
# Colores
for sent in esp_nlp.sents:
    displacy.render(sent, style='ent', jupyter=True)

# Sintagmas

In [ ]:
import spacy
from spacy import displacy
import pandas as pd

nlp_es = spacy.load('es_core_news_lg')

def get_sint_nom(doc):
    df=pd.DataFrame(columns=["Sintagma", "Nucleo"])

    for chunk in doc.noun_chunks:
        fila={"Sintagma":chunk.text, "Nucleo":chunk.root.text}
        df = df.append(fila, ignore_index=True)
    return(df)

# Matriz de similitud coseno

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

lista=list(doc_definitivo['texto'])

def similitudes(texto):
    TFIDF_vectorizer = TfidfVectorizer()
    TFIDF_array=TFIDF_vectorizer.fit_transform(texto).toarray()
    vocab_TFIDF = TFIDF_vectorizer.get_feature_names()
    TFIDF=pd.DataFrame(TFIDF_array, columns=vocab_TFIDF)
    similarity_matrix = cosine_similarity(TFIDF_array)
    similarity_df = pd.DataFrame(similarity_matrix)
    return(similarity_df)

similitudes(lista)

# Modelos de clasificación (Bayesiano, XGBoost, SVM)

In [ ]:
import pandas as pd
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

modellist=['Bayesiano Ingenuo', 'SVC', 'XGBoost']
disp_labels=['barato', 'Medio','Caro']

def run_models(texto_tfidf, label, modellist, disp_labels = None):
    """ 
        Este funcion entrena cada modelo definido en lista input, separa entre train y test,
        predice las clases para el conjunto test y calcula el accuracy de entrenamiento y test,
        asi como la matriz de confusion par cada modelo.
        
        Input: 
            texto_tfidf: matriz de TFIDF del Corpus
            label: columna con el label de la clase de cada documento. Tienen que ser numéricas
            modellist: lista de strings con los modelos a entrenar
            disp_labels: labels de las clases para las matrices de confusion
            
        Output:
            matriz de confusion, accuacy de entrenaiento, test y fiabilidad para cada modelo entrenado
    """
    plt.rcParams["figure.figsize"] = (20,3)
    
    texto_train, texto_test, label_train, label_test = train_test_split(texto_tfidf, label, train_size = 0.8,\
                                                                        random_state=100, stratify=label)
    
    for name in modellist:
        if name == "Bayesiano Ingenuo":
            model = MultinomialNB()
        if name == "SVC":
            model = SVC(kernel='linear')
        if name == "XGBoost":
            model = XGBClassifier(n_jobs = -1, random_state = 100)


        model.fit(texto_train, label_train)
        categs_pred = model.predict(texto_test)
        acc_train = model.score(texto_train, label_train)
        acc_test = model.score(texto_test, label_test)
        ConfusionMatrixDisplay.from_predictions(label_test, categs_pred, display_labels = disp_labels);
        

        # Confusion Matrix
        print('Para el modelo','\033[1m' + str(name) + '\033[0m',"matriz de confusion es:")
        plt.show()
        print("Con un accuracy de train de:", round(acc_train*100,2),"%")
        print("Con un accuracy de test de:", round(acc_test*100,2),"%")
        print("Con una fiabilidad de:", round(acc_test/acc_train*100,2),"%", "\n")

# Clasificación con transformers

In [ ]:
from transformers import pipeline

## Análisis de sentimientos
model = "finiteautomata/beto-sentiment-analysis"
sentiment_pipeline = pipeline("sentiment-analysis", model=model)
print(sentiment_pipeline(["Un sensacional partido de baloncesto", "la peor película premiada en la gala"]))

## Natural Language Inference (¿Está contenida la respuesta a esta pregunta??)
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/qnli-electra-base')
score = model.predict([("Where is the capital of France?","Paris is the capital of France.")])
print(score)

## Zero-Shot-Classification
model = "Recognai/zeroshot_selectra_medium"
clasif = pipeline("zero-shot-classification", model=model)
print(clasif(
    "El IPC sube de golpe y se dispara hasta el 9,8% en marzo, su tasa más alta desde 1985",
    candidate_labels=["política", "deportes", "economía"]))

# QA con transformers

In [ ]:
from transformers import pipeline

model = "mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es"

qa_pipeline = pipeline("question-answering", model=model)

Contexto = "La montaña mágica (Der Zauberberg, en el original alemán) es una novela de Thomas Mann que se publicó " \
"en 1924. Es considerada la novela más importante de su autor y un clásico de la literatura en lengua alemana" \
"del siglo XX que ha sido traducido a numerosos idiomas, siendo dominio público en países como Estados Unidos, " \
"España, Brasil, entre otros. " \
"Thomas Mann comenzó a escribir la novela en 1912, a raíz de una visita a su esposa en el Sanatorio Wald de Davos " \
"en el que se encontraba internada. La concibió inicialmente como una novela corta, pero el proyecto fue creciendo " 
"con el tiempo hasta convertirse en una obra mucho más extensa. La obra narra la estancia de su protagonista principal, " \
"el joven Hans Castorp, en un sanatorio de los Alpes suizos al que inicialmente había llegado únicamente como " \
"visitante. La obra ha sido calificada de novela filosófica, porque, aunque se ajusta al molde genérico del " \
"Bildungsroman o novela de aprendizaje, introduce reflexiones sobre los temas más variados, tanto a cargo del narrador" \
"como de los personajes (especialmente Naphta y Settembrini, los encargados de la educación del protagonista)."

print(qa_pipeline(question = "¿Cuándo se escribió la Montaña Magica?", context= Contexto))
print(qa_pipeline(question = "¿Cómo se titula La Montaña Mágica originalmente?", context= Contexto))
print(qa_pipeline(question = "¿En qué idioma está escrita La Montaña Mágica?", context= Contexto))

print(qa_pipeline(question = "Who wrote Der Zauerberg?", context= Contexto))
print(qa_pipeline(question = "Where was Thomas Mann wife staying?", context= Contexto))

# Traducción con transformers

In [ ]:
from transformers import pipeline

model = "Helsinki-NLP/opus-mt-en-es"

translator = pipeline("translation", model=model)

translator("How many roads must a man walk down " \
"Before you call him a man? " \
"How many seas must a white dove sail" \
"Before she sleeps in the sand? " \
"Yes, and how many times must the cannonballs fly " \
"Before they're forever banned?")

# Resúmenes con transformers

In [ ]:
from transformers import EncoderDecoderModel, AutoTokenizer

text = "La montaña mágica (Der Zauberberg, en el original alemán) es una novela de Thomas Mann que se publicó " \
"en 1924. Es considerada la novela más importante de su autor y un clásico de la literatura en lengua alemana" \
"del siglo XX que ha sido traducido a numerosos idiomas, siendo dominio público en países como Estados Unidos, " \
"España, Brasil, entre otros. " \
"Thomas Mann comenzó a escribir la novela en 1912, a raíz de una visita a su esposa en el Sanatorio Wald de Davos " \
"en el que se encontraba internada. La concibió inicialmente como una novela corta, pero el proyecto fue creciendo " 
"con el tiempo hasta convertirse en una obra mucho más extensa. La obra narra la estancia de su protagonista principal, " \
"el joven Hans Castorp, en un sanatorio de los Alpes suizos al que inicialmente había llegado únicamente como " \
"visitante. La obra ha sido calificada de novela filosófica, porque, aunque se ajusta al molde genérico del " \
"Bildungsroman o novela de aprendizaje, introduce reflexiones sobre los temas más variados, tanto a cargo del narrador" \
"como de los personajes (especialmente Naphta y Settembrini, los encargados de la educación del protagonista)."

print("Text:")
print(text)

tokenizer = AutoTokenizer.from_pretrained("IIC/marimari-r2r-mlsum")
model = EncoderDecoderModel.from_pretrained("IIC/marimari-r2r-mlsum")

input_ids = tokenizer(text, return_tensors="pt").input_ids
output_ids = model.generate(input_ids)[0]

print("Resumen:")
print(tokenizer.decode(output_ids, skip_special_tokens=True))

# Generación de texto

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation', model = 'datificate/gpt2-small-spanish')
generator("Hace muchos años em las Tierras Altas de Escocia", max_length = 50, num_return_sequences=3)